Using [lorem-ipsum.pdf.json](./lorem-ipsum.pdf.json)

In [1]:
import os
import json

trace = json.load(open('test_fontArialNotEmbedded.pdf.json'))
#trace = json.load(open('lorem-ipsum.pdf.json'))

nodes = []
edges = []

for run in trace['runs']:
  runid = run['name']
  pid = 0
  clusters = []
  for proc in run['processes']:
      cluster = []
      print(pid, proc['name'],proc['long_name'],proc['parent'])
      procid = '%s_p%s' % (runid, pid)
      nodes.append({
            'data': {
                'id': procid,
                'label': proc['long_name'],
                'classes': 'process'
                }
      })
      cluster.append(procid)
      # Add parent-child relationship:
      if proc['parent']:
          edges.append({
                'data': {
                    'source': '%s_p%s' % (runid, proc['parent'][0]),                
                    'target': '%s_p%s' % (runid, pid),
                    'label': proc['parent'][1],
                    'classes': ['process', 'directed']
                    }
          })
      # And files:
      for i, fin in enumerate(proc['reads']):
          fid = '%s_p%s_r%s' % (runid, pid, i)
          fname = os.path.basename(fin)
          nodes.append({
                'data': {
                    'id': fid,
                    'label': fname,
                    'path': fin,
#                    'parent': procid,
                    'classes': 'file'
                    }
          })
          cluster.append(fid)
          edges.append({
                'data': {
                    'source': fid,                
                    'target': '%s_p%s' % (runid, pid),
                    'label': 'reads',
                    'classes': ['read', 'directed']
                    }
          })
      for i, fin in enumerate(proc['writes']):
          fid = '%s_p%s_w%s' % (runid, pid, i)
          fname = os.path.basename(fin)
          nodes.append({
                'data': {
                    'id': fid,
                    'label': fname,
                    'path': fin,
 #                   'parent': procid,
                    'classes': 'file'
                    }
          })
          cluster.append(fid)
          edges.append({
                'data': {
                    'source': '%s_p%s' % (runid, pid),
                    'target': fid,                
                    'label': 'writes',
                    'classes': ['write', 'directed']
                    }
          })

      # Next:
      clusters.append(cluster)
      pid += 1

data = {'nodes': nodes, 'edges': edges }
print(nodes[0])

0 1 gs (1) None
{'data': {'id': 'run0_p0', 'label': 'gs (1)', 'classes': 'process'}}


In [14]:
import ipycytoscape
import networkx as nx
from ipywidgets import Layout

cytoscapeobj = ipycytoscape.CytoscapeWidget(layout=Layout(width='100%', height='800px', margin='0px auto 0px auto'))
cytoscapeobj.graph.add_graph_from_json(data)
cytoscapeobj.set_style([
                    {
                        'selector': 'node',
                        'css': {
                            'font-size': '32px',
                            'width': '16px',
                            'height': '16px',
                            'content': '',
                            'text-valign': 'center',
                            'background-color': 'lightgray',
                            'text-outline-width': 2,
                            'text-outline-color': 'white',
                            "border-width": "2px",
                            "border-color": "black",
                            "border-opacity": "1.0",
                        }
                    },
                    {
                        'selector': 'node[classes="process"]',
                        'css': {
#                            'content': 'data(label)',
                            'width': '32px',
                            'height': '32px',
                            'background-color': 'dimgray',
                        }
                    },
                    {
                        'selector': 'node[classes="file"]',
                        'css': {
                            'content': '',
                        }
                    },
                    {
                        'selector': 'node:active',
                        'css': {
                            'content': 'data(label)',
                        }
                    },
                    {
                        'selector': 'edge',
                        'style': {
                            'width': 3,
                            'color': 'white',
                            'text-valign': 'center',
                            'text-outline-width': 2,
                            'text-outline-color': 'black',
                            'line-color': 'black',
                            'opacity': '0.5',
                            'target-arrow-shape': 'triangle',
                            'target-arrow-color': 'black',
                            'curve-style': 'bezier',
#                            'content': 'data(label)',
                            'font-size': '10px'
                        }
                    },
                    {
                        'selector': 'edge[label="reads"]',
                        'style': {
                            'text-outline-color': 'darkblue',
                            'line-color': 'darkblue',
                            'target-arrow-color': 'darkblue'
                        }
                    },
                    {
                        'selector': 'edge[label="writes"]',
                        'style': {
                            'text-outline-color': 'darkred',
                            'line-color': 'darkred',
                            'target-arrow-color': 'darkred',
                            'opacity': '1.0',
                            'width': 5
                        }
                    }
])
#cytoscapeobj.set_layout(name='cola', flow={ 'axis': 'x', 'minSeparation': 75 }, nodeDimensionsIncludeLabels=True, maxSimulationTime = 1000000, avoidOverlap=False, randomize=False )
#cytoscapeobj.set_layout(name='breadthfirst', directed=True, circle=False, nodeDimensionsIncludeLabels=True, spacingFactor=0.7)
cytoscapeobj.set_layout(name='breadthfirst', directed=True, circle=True, nodeDimensionsIncludeLabels=True, spacingFactor=4.0)
#cytoscapeobj.set_layout(name='dagre', rankDir='LR', rankSep=1000, nodeSep=1, edgeSep=1, padding=0, nodeDimensionsIncludeLabels=True)
#cytoscapeobj.set_layout(name='klay', klay = { 'spacing': 5, 'direction': 'RIGHT' } )
cytoscapeobj

CytoscapeWidget(cytoscape_layout={'name': 'breadthfirst', 'directed': True, 'circle': True, 'nodeDimensionsInc…